In [27]:
import requests
from bs4 import BeautifulSoup
import csv
import time
import random

In [17]:
# get top3 players from each team

teams = ['ATL', 'BOS', 'BRK', 'CHI', 'CHO', 'CLE', 'DAL', 'DEN', 'DET', 'GSW', 'HOU', 'IND', 'LAC', 'LAL', 'MEM', 'MIA', 'MIL', 'MIN', 'NOP', 'NYK', 'OKC', 'ORL', 'PHI', 'PHO', 'POR', 'SAC', 'SAS', 'TOR', 'UTA', 'WAS']


In [19]:
all_rows = []

for team_abbrev in teams:
    search_url = f'https://www.basketball-reference.com/teams/{team_abbrev}/2023.html'

    response = requests.get(search_url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')

        # Find the table with the specified caption
        table_caption = soup.find('caption', text='Per Game Table')

        if table_caption:
            # Get the parent table element
            table = table_caption.find_parent('table')

            # Extract data from the thead
            headers = [header.text.strip() for header in table.find('thead').find_all(['th'])]

            # Extract data from the tbody
            rows = []
            for row in table.find('tbody').find_all('tr')[:3]:  # Collect the first three entries
                player_column = row.find('td', class_='left')  # Assuming the class of the player column is 'left'
                player_link = player_column.find('a')['href'] if player_column and player_column.find('a') else None
                row_data = [data.text.strip() for data in row.find_all(['td', 'th'])]
                rows.append({'team': team_abbrev, 'player_link': player_link, 'data': row_data})

            # Add the rows to the list of all rows
            all_rows.extend(rows)

        else:
            print("Table caption not found.")
    else:
        print("Failed to fetch the webpage. Status code:", response.status_code)

# Save all data to a single CSV file
csv_filename = 'top_three_players_by_team.csv'
with open(csv_filename, 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)

    # Write the header
    csv_writer.writerow(['team'] + ['player_link'] + headers)

    # Write the data rows
    for row in all_rows:
        csv_writer.writerow([row['team']]+[row['player_link']] + row['data'])

print(f"All data has been exported to {csv_filename}")

/var/folders/0p/t55f5t150hv1sqqwhrm_md340000gn/T/ipykernel_2304/2047657456.py:12: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  table_caption = soup.find('caption', text='Per Game Table')


All data has been exported to top_three_players_by_team.csv


In [20]:
csv_filename = 'top_three_players_by_team.csv'

# Initialize empty lists to store teams and player_links
teams = []
player_links = []

with open(csv_filename, 'r', newline='') as csvfile:
    csv_reader = csv.reader(csvfile)

    # Read the header to get the index of 'team' and 'player_link' columns
    header = next(csv_reader)
    team_index = header.index('team') if 'team' in header else None
    player_link_index = header.index('player_link') if 'player_link' in header else None

    if team_index is not None and player_link_index is not None:
        # Iterate through the rows
        for row in csv_reader:
            # Extract 'team' and 'player_link' from each row
            team = row[team_index] if team_index < len(row) else None
            player_link = row[player_link_index] if player_link_index < len(row) else None

            # Append to the lists
            teams.append(team)
            player_links.append(player_link)

In [28]:
base_url = "https://www.basketball-reference.com"

all_player_rows = []

delay_between_requests = 2  # Set a delay in seconds

runs = 1
# Print or use the extracted teams and player_links as needed
for team, player_link in zip(teams, player_links):

    runs -= 1

    if runs < 0:
        break

    player_url = base_url + player_link.strip(".html") + "/gamelog/2023"

    user_agents = [
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:89.0) Gecko/20100101 Firefox/89.0',
        'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.1.1 Safari/605.1.15',
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Edge/91.0.864.59 Safari/537.36',
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 YaBrowser/21.6.2.855 Yowser/2.5 Safari/537.36',
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 OPR/77.0.4054.275 Safari/537.36',
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Whale/2.10.122.23 Safari/537.36',
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Brave/91.1.25.68 Safari/537.36',
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Vivaldi/4.0.2312.33 Safari/537.36'
    ]


    headers = {'User-Agent': random.choice(user_agents)}


    response = requests.get(player_url, headers=headers)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')

        # Find the table with the specified caption
        table_caption = soup.find('caption', text='2022-23 Regular Season Table')

        if table_caption:
            # Get the parent table element
            table = table_caption.find_parent('table')

            # Extract data from the thead
            headers = [header.text.strip() for header in table.find('thead').find_all('th')]

            # Extract data from the tbody
            rows = []
            table_rows = table.find('tbody').find_all('tr')
            for row in table_rows:
                row_class = row.get('class', [])
                if 'thead' not in row_class:
                    row_data = [data.text.strip() for data in row.find_all(['td', 'th'])]  # Include 'th' elements as well
                    rows.append({'team': team_abbrev, 'data': row_data})

            # Add the rows to the list of all rows
            all_player_rows.extend(rows)

        else:
            print("Table caption not found.")
    else:
        print("Failed to fetch the webpage. Status code:", response.status_code)

    time.sleep(delay_between_requests)

Failed to fetch the webpage. Status code: 429


In [21]:
# Save all data to a single CSV file
csv_filename = 'player_season_data_by_team.csv'
with open(csv_filename, 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)

    # Write the header
    csv_writer.writerow(['team'] + headers)

    # Write the data rows
    for row in all_rows:
        csv_writer.writerow([row['team']]+ row['data'])

print(f"All data has been exported to {csv_filename}")

Table caption not found.


/var/folders/0p/t55f5t150hv1sqqwhrm_md340000gn/T/ipykernel_2304/3551388406.py:5: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  table_caption = soup.find('caption', text='2022-23 Regular Season Table')
